<a href="https://colab.research.google.com/github/letun1987/Tinkoff-Data-analysis-in-the-industry-2023/blob/main/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%B2_%D0%B8%D0%BD%D0%B4%D1%83%D1%81%D1%82%D1%80%D0%B8%D0%B8_%D0%9F%D1%80%D0%BE%D0%B5%D0%BA%D1%82_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import psycopg2
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Создадим соединение с базой данных.

In [ ]:
connection = psycopg2.connect(user="student",
                                  password="JvLda93aA",
                                  host="158.160.52.106",
                                  port="5432",
                                  database="postgres",
                                  options="-c search_path=msu_analytics")

Получаем данные из базы согласно фильтрам, в срезе времени - месяца.

In [ ]:
df_client_month = pd.read_sql("""select
        date_trunc('month', visit_dttm::date) as month,
        COUNT(DISTINCT c.client_rk) as clients,
        round((COUNT(DISTINCT a.account_rk)::numeric/COUNT(DISTINCT c.client_rk)::numeric)*100,1) as customers,
        round((COUNT(DISTINCT a2.account_rk)::numeric/COUNT(DISTINCT c.client_rk)::numeric)*100,1) as applicants,
        round(((COUNT(DISTINCT a.account_rk) filter (where g.game_flg =1))::numeric/COUNT(DISTINCT c.client_rk)::numeric)*100,1) as players
from client c
LEFT JOIN
        account a  ON a.client_rk = c.client_rk
LEFT JOIN
        application a2  on a2.account_rk  = a.account_rk
LEFT JOIN
        game g  on g.game_rk  = a2.game_rk
GROUP BY 1
order by 1""", connection)
df_client_month

<ipython-input-3-586426152e4d>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_client_month = pd.read_sql("""select


,month,clients,customers,applicants,players
0,2022-08-31 21:00:00+00:00,778,24.4,6.6,5.0
1,2022-09-30 21:00:00+00:00,843,25.7,7.7,6.6
2,2022-10-31 21:00:00+00:00,786,25.4,7.1,6.4
3,2022-11-30 21:00:00+00:00,805,24.3,7.1,4.1
4,2022-12-31 21:00:00+00:00,852,25.0,7.5,2.3
5,2023-01-31 21:00:00+00:00,32,25.0,0.0,0.0


In [ ]:
df_client_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   month       6 non-null      datetime64[ns, UTC]
 1   clients     6 non-null      int64              
 2   customers   6 non-null      float64            
 3   applicants  6 non-null      float64            
 4   players     6 non-null      float64            
dtypes: datetime64[ns, UTC](1), float64(3), int64(1)
memory usage: 368.0 bytes


Сделаем визуализацию наших данных, посмотрим как они выглядят в динамике. Совместим на одно графике количество посетителей и их конверсию в зависимости от их действий на нашем сайте.

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(
    go.Bar(x=df_client_month['month'], y=df_client_month['clients'].values, name="Посетители сайта", marker_color = 'rgb(158,202,225)',
                                                                                            marker_line_color='rgb(8,48,107)',
                                                                                            marker_line_width=1.5, opacity=0.6),

    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_client_month['month'], y=df_client_month['customers'], name="Зарегистрировались на сайте"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=df_client_month['month'], y=df_client_month['applicants'], name="Оставили заявку на игру"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=df_client_month['month'], y=df_client_month['players'], name="Пришли на игру"),
    secondary_y=True,
)


fig.update_layout(
    title_text="<b>Динамика по месяцам<b>",
)


fig.update_xaxes(title_text="<b>Месяц<b>")


fig.update_yaxes(title_text="<b>Количество посетителей сайта, чел.</b>", secondary_y=False, showgrid=False)
fig.update_yaxes(title_text="<b>Конверсия, %</b>", secondary_y=True, showgrid=False)

fig.show()

Теперь визуализируем эти данные на недельном графике.

In [ ]:
df_client_week = pd.read_sql("""select
        date_trunc('week', visit_dttm::date) as week,
        COUNT(DISTINCT c.client_rk) as clients,
        round((COUNT(DISTINCT a.account_rk)::numeric/COUNT(DISTINCT c.client_rk)::numeric)*100,1) as customers,
        round((COUNT(DISTINCT a2.account_rk)::numeric/COUNT(DISTINCT c.client_rk)::numeric)*100,1) as applicants,
        round(((COUNT(DISTINCT a.account_rk) filter (where g.game_flg =1))::numeric/COUNT(DISTINCT c.client_rk)::numeric)*100,1) as players
from client c
LEFT JOIN
        account a  ON a.client_rk = c.client_rk
LEFT JOIN
        application a2  on a2.account_rk  = a.account_rk
LEFT JOIN
        game g  on g.game_rk  = a2.game_rk
GROUP BY 1
order by 1""", connection)
df_client_week

/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,week,clients,customers,applicants,players
0,2022-08-28 21:00:00+00:00,111,28.8,6.3,4.5
1,2022-09-04 21:00:00+00:00,184,16.8,3.3,2.7
2,2022-09-11 21:00:00+00:00,167,25.7,9.6,4.8
3,2022-09-18 21:00:00+00:00,181,24.9,6.6,6.1
4,2022-09-25 21:00:00+00:00,195,30.3,8.7,8.7
5,2022-10-02 21:00:00+00:00,204,28.4,8.8,7.4
6,2022-10-09 21:00:00+00:00,190,23.2,6.8,6.8
7,2022-10-16 21:00:00+00:00,171,24.0,7.0,5.8
8,2022-10-23 21:00:00+00:00,189,25.4,7.9,5.8
9,2022-10-30 21:00:00+00:00,197,23.9,8.1,7.1


In [ ]:
df_client_week.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   week        23 non-null     datetime64[ns, UTC]
 1   clients     23 non-null     int64              
 2   customers   23 non-null     float64            
 3   applicants  23 non-null     float64            
 4   players     23 non-null     float64            
dtypes: datetime64[ns, UTC](1), float64(3), int64(1)
memory usage: 1.0 KB


In [ ]:
df_client_week.head()

,week,clients,customers,applicants,players
0,2022-08-28 21:00:00+00:00,111,28.8,6.3,4.5
1,2022-09-04 21:00:00+00:00,184,16.8,3.3,2.7
2,2022-09-11 21:00:00+00:00,167,25.7,9.6,4.8
3,2022-09-18 21:00:00+00:00,181,24.9,6.6,6.1
4,2022-09-25 21:00:00+00:00,195,30.3,8.7,8.7


In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(
    go.Bar(x=df_client_week['week'], y=df_client_week['clients'].values, name="Посетители сайта", marker_color = 'rgb(158,202,225)',
                                                                                            marker_line_color='rgb(8,48,107)',
                                                                                            marker_line_width=1.5, opacity=0.6),

    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_client_week['week'], y=df_client_week['customers'], name="Зарегистрировались на сайте"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=df_client_week['week'], y=df_client_week['applicants'], name="Оставили заявку на игру"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=df_client_week['week'], y=df_client_week['players'], name="Пришли на игру"),
    secondary_y=True,
)


fig.update_layout(
    title_text="<b>Динамика по неделям<b>",
)


fig.update_xaxes(title_text="<b>Недели<b>")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Количество посетителей сайта, чел.</b>", secondary_y=False, showgrid=False)
fig.update_yaxes(title_text="<b>Конверсия, %</b>", secondary_y=True, showgrid=False)

fig.show()

In [ ]:
df_client_month.describe()

,clients,customers,applicants,players
count,6.000000,6.000000,6.000000,6.000000
mean,682.666667,24.966667,6.000000,4.066667
std,320.149757,0.546504,2.963781,2.545323
min,32.000000,24.300000,0.000000,0.000000
25%,780.000000,24.550000,6.725000,2.750000
50%,795.500000,25.000000,7.100000,4.550000
75%,833.500000,25.300000,7.400000,6.050000
max,852.000000,25.700000,7.700000,6.600000


In [ ]:
df_client_week.describe()

,clients,customers,applicants,players
count,23.000000,23.000000,23.00000,23.000000
mean,178.086957,25.100000,7.10000,4.721739
std,28.471538,2.935209,1.50212,2.306110
min,81.000000,16.800000,3.30000,0.000000
25%,175.000000,23.750000,6.70000,2.750000
50%,182.000000,24.900000,7.10000,4.800000
75%,193.500000,27.200000,8.05000,6.600000
max,205.000000,30.300000,9.60000,8.700000


**Пройдемся сначала по сухим цифрам статистики в разрезе месяцев:**

* Среднее количестов новых клиентов 682 чел.(медиана 795). Доля тех, кто зарегистроровался в среднем 25.28% (медиана 24.85 %), оставили заявку на игру в среднем 6.93 % (медиана 6.9 %), пришли на игру в средем 4.1 % (медиана 4.75%). Разница, обусловленная между средним и медианой по количеству новых клиентов, появляется из-за того, что в феврале мы берем данные всего с двух дней, а предыдущие месяцы расчитывались в полном объеме.
Сравним с цифрами по неделям:
* Среднее количестов новых клиентов 178 чел.(медиана 182). Доля тех, кто зарегистроровался в среднем 25.10% (медиана 24.90 %), оставили заявку на игру в среднем 7.1 % (медиана 7.1 %), пришли на игру в средем 4.7 % (медиана 4.8%).
Как мы видим значение долей клиентов в недельном и месячных срезах практически одинаковые и разница не более +- 1 %. Используя недельный срез данных, а ровно как и график можно более детально увидеть колебания отдельных показателей, например, падение конверсии зарегистрировавшихся пользователей до значения 16 % в первые дни сентября. На месячном графике эти колебания не заметны.\

**Рассмотрим недельный график и сделаем выводы на основании полученных данных:**

* В среднем наш сайт посешает 182 человека в неделю. Но мы наблюдаем спад в первую неделю сентября, можно предположить, что он вызван праздиком 1 сентября (сбор детей в школу, покупка одежды, принадлежностей и прочего), но также стоит учесть, что в первом недельном срезе, нет данных за август, часть августовских дней с пустыми значениями вошла в него. Также есть спад по посещениям в последнюю неделю, но он обусловлен тем, что расчет не охватывает полную неделю, а берется на основании 3-4 дней, соотвественно просто не хватает данных. Что можно сделать: увеличить конверсию можно за счет создания и разработки специальных квестов, которые расчитаны на аудиторию школьного возраста (8-17 лет), возможно создать доплнительные активные игры, на этот период времени. Создать так сказать сезонный квест(активные игры), их можно будет запускать в период школьных каникул, как промо-акции, которые действуют ограниченный период времени в течении года. О проведении этих акций можно напоминать родителям, например, отправляя сообщения на email, sms, telegram. В свою очередь, когда пользователь регистрируется на сайте, в форме регистрации должны быть поле, где пользователь указывает информацию о детях.
* И есть небольшой спад по клиентам в 10-х числах ноября. Возможно он был вызван государственными ноябрьскими праздниками. По всем праздникам как ноябрьским, так и по другим можно сделать специальное предложение, информация о котором будет на нашем сайте, а также рассылаться уже зарегирированным клиентам с напоминанием, что в переди ждут праздники и мы предлогаем интересно и с пользой провести время. Рассылка должна происходить заранее, например, за неделю чтобы пользователь мог подумать, обсудить с коллегами и друзьями, а затем приянть положительное решение. Но и не слишком рано (за 3 недели) иначе человек просто забудет. Очень хорошо сделать акцент на проведение новогодних корпоративов в качестве игры-квеста, это очень хорошо укрепляет командный дух.
* Доля зарегистрировавшихся пользователей составляет примерно 25 % от его посетителей, за исключением падения этого показателя в начале сентября до 16 %. По сути мы имеем стабильную конверсии в 25% зарегистрированных пользователей, иными словами каждый 4-й наш посетитель регистрируется на нашем сайте.
* Практически на протяжении всего графика, можно наблюдать корреляцию между нашими данными при увеличении посещений нашего сайта и процента пользователей которые зарегистрировались, увеличивается и процент бронирований и начала игр. Эта же тенденция наблюдается и в обратном направлении. За исключением последняя неделя января и начало февраля, там мы наблюдаем рост конверсии зарегистрированных пользователей, и падения показателей по бронированию и началу игр.
* Доля клиентов, которые оставили заявку на игру составляет примерно 7.1 % от посетителей сайта и соответственно доля клиентов, которые пришли на игру равна 4.7 %. Здесь мы видим общую тенденцию падения конверсии с 25% до 4.7%. Необходимо детально разобраться какие факторы на это влияют, принять меры для увеличения этих показателей.

**Рекомендации для увеличения конверсии пользователей:**

1. Проверить как написаны описания к нашим квестам, после прочтения появляется ли у пользователя интерес? Отдельно выделить жанр квеста - возможно это horror в духе "Пилы" или "Silent Hill", или примерить на себе роль детектива, как Шерлок Холмс, и разгадать наисложнейшую головоломку. Нельзя забывать и про фотографии, они должны быть в хорошем качестве и отражать обстановку локации, чтобы клиент мог визуально представить, где ему предстоит играть. Можно так же добавить видео из коротких нарезок, как проходят квесты другие участники.
2. На игру регистрируется 7%, а приходит только 4% - возможно клиенту просто не напомнили, что у него завтра игра и он забыл. Необходимо сделать систему уведомлений.
3. Проверить работу сайта, меню навигации, все перемешения от регистрации до бронирования должны быть интуитивно понятными и каждый раздел можно легко найти.
4. В случае отказа клиента от игры, постараться выяснить в чем причина и предложить перенести игру на другое время или локацию, когда клиенту удобно.
5. Если клиент хочет забронировать игру, а это время и место занято, то всегда можно предложить другого партнера, который есть в этом городе.
6. Провести анализ квестов - выявить какие самые популярные, а какие наоборот вообще не пользуються спросом. Понять в чем причины отсутствия спроса - это недоработка сотрудников или квест слишком сложный, неинтересный. Попытаться улучшить квест, если это невозможно, то отказаться от него и заменить другим.
7. Проверить эффективность штатного расписания квестов, режимов работы партнеров. Необходимо составить эффективное расписание работы.
8. Постоянный мониторинг конкурентов - их цены, квесты, предложения. Возможно мы просто не в рынке по цене, или появился какой-то новый, более интересный продукт.
9. После прохождения квестов получать обратную связь от клиентов. Что понравилось, не понравилось, общие рекомендации по улучшению самого квеста. Так мы сможем выявить общие проблемы в работе, на которые будут указывать большинство пользователей.
10. Создать программу лояльность, как для постоянных клиентов, чтобы их удерживать, так и для новых, например, при регистрации получение скидки на первый квест 20%, тем самым мы увеличим конверсию как из зарегистрировавшихся пользователей, так и тех, кто забронирует игру, чтобы использовать свой промо-код на 20%. Дополнительно, при регистрации клиент оставит свой email и телефон, а это канал связи, и мы всегда сможем напомнить о себе.
11. Если есть возможность, то подключить первичную проверку геолокации пользователя при открытии сайта, т.к. наши партнеры открыты в разных городах.
12. Специальные предложения и промо-акции, на сайте, в общем списке меню должны выделяться отдельным цветом, чтобы взгляд пользователя тут же зацеплялся за них. Внутри самих акций можно использовать таймеры, чтобы клиент видел, что предложение ограничено по времени.
13. Информация о стоимости должна быть видна и понятна для пользователя. Исключить варианты "Пишите в директ" и "Позвоните для уточнения".
14. Необходимо ввести систему регламента и мониторинга работы сотрудников на местах. Причины снижения конверсии пользователей, могут быть связаны с некачественной работой сотрудников (дали неверную информацию или в неполном объеме, не предложили, например, клиенту провести игру у другого партнера, если по каким-то причинам он хочет отказаться от игры и т.д.). Конечно и система мотивации сотрудников, должна работать и мотивировать их - больше клиентов, то и зарплата больше.

